# Extract Features

1. Read csv files and load as dfs
2. Combine dfs
3. Get semantic cosine similarity

In [ ]:
import os, sys

import pandas as pd

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

import log_files
from log_files import LogData
from data_processing import DataProcessing
from feature_extraction import SpacyFeatureExtraction

In [ ]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Predictions

- Use the structure from `1-generate_predictions-all_domains.ipynb`

In [ ]:
log_file_path = "data/prediction_logs"
predictions = True
predictions_df = log_files.read_data(notebook_dir, log_file_path, predictions)
predictions_df.head(7)

## Observations

In [ ]:
log_file_path = "data/observation_logs"
predictions = False
observations_df = log_files.read_data(notebook_dir, log_file_path, predictions)
observations_df.head(7)

## Both

- Create a knowledge graph
    - Nodes: words
    - Edges: connection to other words (same/diff sentence)
- Look at code from Graphbreeding project on 2019 Mac

In [ ]:
df = DataProcessing.concat_dfs([predictions_df, observations_df])
df.head(3)

In [ ]:
predictions = DataProcessing.df_to_list(predictions_df, "Base Sentence")
observations = DataProcessing.df_to_list(observations_df, "Base Sentence")

In [ ]:
# from text_generation_models import LlamaVersatileTextGenerationModel
# llama_versatile_generation_model = LlamaVersatileTextGenerationModel()

# from text_generation_models import TextGenerationModelFactory
# tgmf = TextGenerationModelFactory()

# # llama_versatile_generation_model = tgmf.create_instance(model_name='llama-3.3-70b-versatile')
# # llama_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant')
# llama_8b_8192_generation_model = tgmf.create_instance('llama3-8b-8192')


# prompt = f"Can you return the observations ({observations}) that certify this prediction ({predictions[0]})? Only write the observations that certify the prediction."
# # prompt = f"Can you return a list of the observations ({observations}) that certify this prediction ({predictions[0]}) and why? Only write the observations that certify the prediction and why. Do not write any other text. "
# input_prompt = llama_8b_8192_generation_model.user(prompt)
# # print(input_prompt)
# # raw_text = self.chat_completion([self.user(prompt_template)])
# raw_text = llama_8b_8192_generation_model.chat_completion([input_prompt])
# print(raw_text)

In [ ]:
# raw_text

In [ ]:
# # Extract observations from the string
# observations = raw_text.split('\n\n')[1].split('\n')[1:-1]
# # Remove numbering and quotes
# observations = [obs.split('. ', 1)[1].strip('"') for obs in observations]
# # Print the list
# print(observations)


In [ ]:
# type(observations)

In [ ]:
# observations[1]

In [ ]:
# from text_generation_models import TextGenerationModelFactory
# tgmf = TextGenerationModelFactory()

# llama_versatile_generation_model = tgmf.create_instance(model_name='llama-3.3-70b-versatile')
# llama_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant')
# llama_70b_8192_generation_model = tgmf.create_instance('llama3-70b-8192')
# llama_8b_8192_generation_model = tgmf.create_instance('llama3-8b-8192')

# models = [llama_70b_8192_generation_model]
# # models = [llama_versatile_generation_model, llama_instant_generation_model, llama_70b_8192_generation_model, llama_8b_8192_generation_model]
# # Prompt for the model

# prompt = f"Can you return the observations ({observations}) that certify this prediction ({predictions[0]})? Write in the format of ({predictions[0]}, {observations})"
# input_prompt = llama_versatile_generation_model.user(prompt)
# # print(input_prompt)

# model_certify = {}
# for model in models:    
    
#     raw_text = model.chat_completion([input_prompt])
#     print(raw_text)

In [ ]:
# df

In [ ]:
# data = []
# for model, output in model_certify.items():
#     for o in output:
#         data.append([model, o])

# # Create the DataFrame
# df = pd.DataFrame(data, columns=['Model', 'Output'])
# df

In [ ]:
# import pandas as pd
# from text_generation_models import TextGenerationModelFactory

# # Initialize the TextGenerationModelFactory
# tgmf = TextGenerationModelFactory()

# # Create instances of the models
# llama_versatile_generation_model = tgmf.create_instance(model_name='llama-3.3-70b-versatile')
# llama_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant')
# llama_70b_8192_generation_model = tgmf.create_instance('llama3-70b-8192')
# llama_8b_8192_generation_model = tgmf.create_instance('llama3-8b-8192')

# # List of models
# models = [llama_instant_generation_model, llama_70b_8192_generation_model, llama_8b_8192_generation_model]

# # Prompt for the model
# prompt = f"Can you return a list of the observations ({observations}) that certify this prediction ({predictions[0]})? Only write the observations that certify the prediction. Do not write any other text."
# input_prompt = llama_versatile_generation_model.user(prompt)

# # Dictionary to store model outputs
# model_certify = {}
# for model in models:
#     raw_text = model.chat_completion([input_prompt])
#     output = [line.strip().replace("*", "") for line in raw_text.split("\n") if line.strip()]
#     model_certify[model.model_name] = output

# # Prepare data for DataFrame
# data = []
# for model, output in model_certify.items():
#     for o in output:
#         if isinstance(o, list):
#             o = ', '.join(o)
#         data.append([predictions[0], model, o])

# # Create the DataFrame
# df = pd.DataFrame(data, columns=['Prediction', 'Model', 'Observations'])

# # Display the DataFrame
# df


In [ ]:
# df

- Don't use only the {predictions[0]}, use the structure from `1-generate_prediction-all_domains.ipynb` and the spacy tokens

In [ ]:
{predictions[0]}

In [ ]:
from text_generation_models import TextGenerationModelFactory
tgmf = TextGenerationModelFactory()

llama_versatile_generation_model = tgmf.create_instance(model_name='llama-3.3-70b-versatile')
llama_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant')
llama_70b_8192_generation_model = tgmf.create_instance('llama3-70b-8192')
llama_8b_8192_generation_model = tgmf.create_instance('llama3-8b-8192')

# models = [llama_instant_generation_model, llama_70b_8192_generation_model, llama_8b_8192_generation_model]
models = [llama_instant_generation_model]

# Prompt for the model

prompt = f"Return a list of the observations ({observations}) that certify this prediction ({predictions[0]})?"
input_prompt = llama_versatile_generation_model.user(prompt)
# print(input_prompt)

# df = pd.DataFrame(columns=["Model", "Prompt", "Response"])
model_certify = {}
for model in models:    
    
    raw_text = model.chat_completion([input_prompt])
    output = []
    for line in raw_text.split("\n"):
        if line.strip():  # Skip empty lines
            output.append(line.strip())
    # print(output)
    model_certify[model.model_name] = output

print(model_certify)
model_certify.keys()

data = []
for model, output in model_certify.items():
    for output in output:
        data.append([model, output])

# Create the DataFrame
df = pd.DataFrame(data, columns=['Model', 'Output'])
df

In [ ]:


# https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3
from transformers import pipeline

In [ ]:

prompt = f"Return top 5 list of the observations ({observations}) that certify this prediction ({predictions[0]})?"

messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": prompt},
]
chatbot = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.3")
chatbot(messages)